<a href="https://colab.research.google.com/github/cevparlak/CARFAC-24/blob/main/carfac24_cnn_lstm_cc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
# !pip install keras-self-attention

Mounted at /content/gdrive


In [ ]:
import keras
import sys
import tensorflow as tf
print(tf.__version__)
print(keras.__version__)
print(sys.version)
!pip install -qq -U tensorflow-addons

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
from __future__ import print_function
## Package
import tensorflow as tf

import IPython.display as ipd
import librosa
# import librosa.display
import numpy as np
import pandas as pd
from pandas import DataFrame

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.collections import QuadMesh
import plotly.graph_objs as go
import plotly.offline as py
import plotly.tools as tls
import seaborn as sn
import scipy.io.wavfile
py.init_notebook_mode(connected=True)

from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from tensorflow.keras.callbacks import  History, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Reshape, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, GRU, Bidirectional
from tensorflow.keras.layers import Conv2D, MaxPooling2D,TimeDistributed

from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers as L

# from keras_self_attention import SeqSelfAttention
# from attention import Attention

import scipy.io as sio
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from scipy.io import arff
## Python
import random as rn
import sys
from sklearn import preprocessing
import glob
import os

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# imports
# import os
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"
# import my_models

model_list1=['m_nv','nv','sVGG','VGG16','VGG19','Res v2','Res v1','CNN_LSTM','CNN_LSTM2','CNN_LSTM_Att']
alist  = pd.DataFrame(columns=['Model','Train', 'Test', 'Acc', 'Loss','Precision','Recall','F1','Kappa','B_Accuracy'])
alist2 = pd.DataFrame(columns=['Model','Train', 'Test', 'Acc', 'Loss'])
alist3 = pd.DataFrame(columns=['Model','Train', 'Test', 'Acc', 'Loss'])
best_acc=0
best_loss=1000000000;


nrows=44  # number of rows of feature vectors, columns are auto-adjusted
batch_size1 = 16
epoch1 = 50
val_size1=0.2
test_size1=0.30
hidden_units1=512
hidden_units2=256
lr1=0.001;
factor1=0.90
patience1=10
acf='relu'
n1=12 # for resnet
modelstr1='cnn_'+str(lr1)+'_'+str(factor1)+'_'+str(patience1)+'_'+str(nrows)+'_'+str(epoch1)+'_'+str(batch_size1)+'_'+acf

i=1
# np.random.seed(1234)
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1 )
# sess = tf.compat.v1.Session( graph=tf.compat.v1.get_default_graph(), config=session_conf )
# tf.compat.v1.keras.backend.set_session(sess)

#-----------------------------Keras reproducibility------------------#
# SEED = 1234

# tf.set_random_seed(SEED)
# os.environ['PYTHONHASHSEED'] = str(SEED)
# np.random.seed(SEED)
# rn.seed(SEED)

# session_conf = tf.ConfigProto(
#     intra_op_parallelism_threads=1,
#     inter_op_parallelism_threads=1
# )
# sess = tf.Session(
#     graph=tf.get_default_graph(),
#     config=session_conf
# )
# K.set_session(sess)
#-----------------------------------------------------------------#
list1=glob.glob("E:\\Research\\carfac\\data\\*.npy")
path1="E:\\Research\\carfac\\data\\" # data files directory in local computer
path1="/content/gdrive/My Drive/carfac/a/"  # data files directory in GDrive

list1=[
'1  2  4  5  7  8  9_3  6_nemo train carfac_emp3 _d __cons_512_256_0.3__f0.npy',
'1  2  4  5  7  8  9_3  6_nemo train mel_emp3 _d __cons_512_256_0.3__f0.npy',
'1  2  4  5  7  8  9_3  6_nemo train mfcc_emp3 _d __cons_512_256_0.3__f0.npy',
'63  61  34  20   6  53  40  39   0  27  35   5_ased train carfac_emp3 _d __cons_512_256_0.3__f0.npy',
'63  61  34  20   6  53  40  39   0  27  35   5_ased train mel_emp3 _d __cons_512_256_0.3_f0.npy',
'63  61  34  20   6  53  40  39   0  27  35   5_ased train mfcc_emp3 _d __cons_512_256_0.3_f0.npy',

]

list2=[
'1  2  4  5  7  8  9_3  6_nemo test carfac_emp3 _d __cons_512_256_0.3__f0.npy',
'1  2  4  5  7  8  9_3  6_nemo test mel_emp3 _d __cons_512_256_0.3__f0.npy',
'1  2  4  5  7  8  9_3  6_nemo test mfcc_emp3 _d __cons_512_256_0.3__f0.npy',
'63  61  34  20   6  53  40  39   0  27  35   5_ased test carfac_emp3 _d __cons_512_256_0.3__f0.npy',
'63  61  34  20   6  53  40  39   0  27  35   5_ased test mel_emp3 _d __cons_512_256_0.3_f0.npy',
'63  61  34  20   6  53  40  39   0  27  35   5_ased test mfcc_emp3 _d __cons_512_256_0.3_f0.npy',

]

z1=0
model_list1=(6) #,0.01,0.1)
for cm3 in range(1):
    # cm=model_list1[cm3]
    cm=5
    batch_list1=(4,8,16,32)
    for cm2 in range(1):
        batch_size1=batch_list1[cm2]
        lr_list1=(0.0001,0.001)
        for cm1 in range(2):
            lr1=lr_list1[cm1]
            # lr1=0.001

            for st in list1:
        #    df=arff.loadarff(st)
                print(path1+st)
                index1 = list1.index(st)
                test_file=list2[index1]
                fname1=list1[index1]
                print(index1)
                print(test_file)

                x=np.load(path1+st)
                y=x[:,-1]
        #    break
                x=x[:,:-1]
        #    break
                x[np.isnan(x)] = 0

                #    x[~np.all(x == 0, axis=1)]
            #    xx=np.any(np.isnan(x))
            # normalize each column independently between [0,1]

                min_max_scaler = preprocessing.MinMaxScaler()
                x= min_max_scaler.fit_transform(x)
                # xx=x
                # del data
                # feature count must be greater than 40 or so for the cnn
                a1=x.shape[1]
                a2=a1 % nrows
                if a2!=0:
                    a2=nrows-a2
                    c1=np.zeros((x.shape[0],a2),dtype=int)
                    x=np.concatenate((x,c1),axis=1)
                values, counts = np.unique(y, return_counts=True)
                n_classes=len(counts)
                ncols=int(x.shape[1]/nrows)
                x_train =x
                y_train=y
                # xxx=x_train
                del x
                del y

                print('x_train shape:', x_train.shape)
                print(x_train.shape[0], 'train samples')

                x_train, x_val, y_train, y_val  = sklearn.model_selection.train_test_split(x_train, y_train, test_size=val_size1, random_state=1)

                # input image dimensions
                img_rows, img_cols = nrows, ncols

                if K.image_data_format() == 'channels_first':
                    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols,1)
                    x_val = x_val.reshape(x_val.shape[0], 1, img_rows, img_cols,1)
                    input_shape = (1, img_rows, img_cols)
                else:
                    x_train = x_train.reshape(x_train.shape[0], 1,img_rows, img_cols, 1)
                    x_val = x_val.reshape(x_val.shape[0], 1,img_rows, img_cols, 1)
                    input_shape = (1,img_rows, img_cols, 1)


                # convert class vectors to binary class matrices
                y_val=y_val-1
                y_val = keras.utils.to_categorical(y_val, n_classes)

                y_train = y_train-1
                y_trainx=y_train
                y_train = keras.utils.to_categorical(y_train, n_classes)
        ###################################
        ###################################
        ###################################
                x=np.load(path1+test_file)
                y=x[:,-1]
        #    break
                x=x[:,:-1]
        #    break
                x[np.isnan(x)] = 0

                #    x[~np.all(x == 0, axis=1)]
            #    xx=np.any(np.isnan(x))
            # normalize each column independently between [0,1]

                min_max_scaler = preprocessing.MinMaxScaler()
                x= min_max_scaler.fit_transform(x)
                # xx=x
                # del data
                # feature count must be greater than 40 or so for the cnn
                a1=x.shape[1]
                a2=a1 % nrows
                if a2!=0:
                    a2=nrows-a2
                    c1=np.zeros((x.shape[0],a2),dtype=int)
                    x=np.concatenate((x,c1),axis=1)
                values, counts = np.unique(y, return_counts=True)
                n_classes=len(counts)
                ncols=int(x.shape[1]/nrows)
                x_test=x
                y_test=y
            #    xxx=x_train
                del x
                del y

                print('x_test shape:', x_test.shape)
                print('y_test shape:', y_test.shape)

                # input image dimensions
                img_rows, img_cols = nrows, ncols


                if K.image_data_format() == 'channels_first':
                    x_test = x_test.reshape(x_test.shape[0], 1, img_rows,img_cols,1)
                    input_shape = (1, img_rows,img_cols,1)
                else:
                    x_test = x_test.reshape(x_test.shape[0], 1,img_rows,img_cols, 1)
                    input_shape = (1,img_rows,img_cols, 1)


                # convert class vectors to binary class matrices
                y_test = y_test-1
                y_testx=y_test
                y_test = keras.utils.to_categorical(y_test, n_classes)

                # np.save("x_train25.npy",x_train)
                # np.save("x_test25.npy",x_test)
                # np.save("y_train25.npy",y_train)
                # np.save("y_test25.npy",y_test)

                cm=20
                # if cm==0:
                #   model, str1 = my_models.mini_nvidia_model(input_shape, n_classes, acf)
                # if cm==1:
                #   model, str1 = my_models.nvidia_model(input_shape, n_classes, acf)
                # if cm==2:
                #   model, str1 = my_models.sVGG(input_shape, n_classes, acf)
                # if cm==3:
                #   model, str1 = my_models.VGG16(input_shape, n_classes, acf)
                # if cm==4:
                #   model, str1 = my_models.VGG19(input_shape, n_classes, acf)
                # if cm==5:
                #   n1=3
                #   model, str1 = my_models.resnet_v2(input_shape, n_classes, acf, n1)
                # if cm==6:
                #   n1=5
                #   model, str1 = my_models.resnet_v1(input_shape, n_classes, acf, n1)
                str1='svgg_lstm_seq_self_att'
                chanDim=-1
                str1='cnn64-128-256lstm512-256-128dnn512'
                inp = Input(shape=(1, img_rows, img_cols, 1))
                x = TimeDistributed(Conv2D(64, (3, 3), padding='same', activation='relu'))(inp)
                x = TimeDistributed(Conv2D(64, (3, 3), padding='same', activation='relu'))(x)
                x = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(x)

                x = TimeDistributed(Conv2D(128, (3, 3), padding='same', activation='relu'))(inp)
                x = TimeDistributed(Conv2D(128, (3, 3), padding='same', activation='relu'))(x)
                x = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(x)

                x = TimeDistributed(Conv2D(256, (3, 3), padding='same', activation='relu'))(inp)
                x = TimeDistributed(Conv2D(256, (3, 3), padding='same', activation='relu'))(x)
                x = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(x)
                x = TimeDistributed(Flatten())(x)
                x = LSTM(units=512, return_sequences=True)(x)
                x = LSTM(units=256, return_sequences=True)(x)
                x = LSTM(units=128, return_sequences=False)(x)
                x = Dropout(0.25)(x)
                x = Flatten()(x)
                x = Dense(512, activation='relu')(x)
                x = Dropout(0.5)(x)
                # x = Dense(512, activation='relu')(x)
                # x = Dropout(0.5)(x)
                x = Dense(n_classes, activation='softmax')(x)
                model = Model(inp, x)
                # model.summary()


                # x = L.Conv2D(64, (3, 3), activation=acf, padding='same')(x)
                # # x = L.BatchNormalization()(x)
                # x = L.Conv2D(64, (3, 3), activation=acf, padding='same')(x)
                # # x = L.BatchNormalization()(x)
                # x = L.MaxPooling2D(pool_size=(2, 2))(x)
                # x = L.Dropout(0.25)(x)

                # x = L.Conv2D(128, (3, 3), activation=acf, padding='same')(x)
                # # x = L.BatchNormalization()(x)
                # x = L.Conv2D(128, (3, 3), activation=acf, padding='same')(x)
                # # x = L.BatchNormalization()(x)
                # x = L.MaxPooling2D(pool_size=(2, 2))(x)
                # x = L.Dropout(0.25)(x)

                # # x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)
                # x = Reshape((-1, 128))(x)
                # print(x.shape)
                # print('xx')
                # # x = L.Bidirectional(LSTM(256, return_sequences=True))(x)  # [b_s, seq_len, vec_dim]
                # # x = SeqSelfAttention(attention_activation ='tanh')(x)
                # # x = L.Bidirectional(LSTM(128, return_sequences=False))(x)  # [b_s, seq_len, vec_dim]

                # # x = L.Bidirectional(LSTM(256, return_sequences=True))(x)  # [b_s, seq_len, vec_dim]
                # # x = L.Bidirectional(LSTM(128, return_sequences=True))(x)  # [b_s, seq_len, vec_dim]

                # x = L.LSTM(512, return_sequences=True)(x)  # [b_s, seq_len, vec_dim]
                # x = L.LSTM(256, return_sequences=True)(x)  # [b_s, seq_len, vec_dim]

                # x = Attention(units=64)(x)
                # x = L.Dense(512, activation=acf)(x)
                # x = Dropout(0.5)(x)

                # x = L.Dense(n_classes, activation='softmax', name='output')(x)
                # model = Model(inp, x)

                # if cm==6:
                #   model, str1 = my_models.ConvSpeechModel(input_shape, n_classes, acf)
                # if cm==7:
                #     model, str1 = my_models.RNNSpeechModel(input_shape, n_classes, acf)
                # if cm==8:
                #     model, str1 = my_models.attRNNSpeechModel(input_shape, n_classes, acf)

                # optimizer1 = keras.optimizers.SGD(lr=lr1, momentum=0.0, decay=0.0, nesterov=True)
                # lr_scheduler = LearningRateScheduler(lr_schedule)
                # lr_reduce2 = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
                # lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=factor1, patience=patience1, min_lr=0.000000001, verbose=1)
                # callbacks = [checkpoint, lr_reduce, lr_scheduler]
                # callbacks = [lr_reduce]
                # opt_name='SGD';

                opt_name='Adam';
                optimizer1=keras.optimizers.Adam(learning_rate=lr1, beta_1=0.9, beta_2=0.999, amsgrad=False)

                model.compile(loss='categorical_crossentropy', optimizer=optimizer1, metrics=['accuracy'])
                head1, tail1 = os.path.split(path1+fname1)
                modelstr1=str1+'_'+str(lr1)+'_'+str(factor1)+'_'+str(patience1)+'_'+str(nrows)+'_'+str(epoch1)+'_'+str(batch_size1)+'_'+acf+'_'+str(test_size1)
                # modelstr2=modelstr1+'_'+opt_name
                modelstr2=modelstr1+'_'+opt_name+'_CC'

                rlrop1 = ReduceLROnPlateau(monitor='loss', factor=factor1, patience=patience1, min_lr=0.000000001, verbose=1)

                history=model.fit(x_train, y_train,
                              epochs=epoch1,
                              validation_data=(x_val, y_val),
                              callbacks=[rlrop1]
                              )
                score = model.evaluate(x_test, y_test, verbose=1)

                #Predict
                print('Test accuracy:', score[1])
                print('Test loss:', score[0])
                # head1, tail1 = os.path.split(path1+st)

            # Plotting the Train Valid Loss Graph
                r1=8
                c1=6
                f1=16
                linew1=3
                plt.clf()
                plt.figure(figsize=(r1,c1))
                plt.rcParams.update({'font.size': f1})
                plt.plot(history.history['loss'],linewidth=linew1)
                plt.plot(history.history['val_loss'],linewidth=linew1)
                plt.title('Model Loss')
                plt.xlabel('epoch')
                plt.ylabel('loss')
                plt.legend(['train', 'test'])
            #    plt.legend(['training', 'test'], loc='upper left')
            #    plt.show()
                plt.savefig(head1+'/b/aloss_'+tail1+'_'+modelstr2+'.png')
                # np.save(head1+'\\aloss_'+tail1+'_'+modelstr2+'.npy', history.history['loss'])
                # np.save(head1+'\\aloss_val_'+tail1+'_'+modelstr2+'.npy', history.history['val_loss'])

                plt.clf()
                plt.figure(figsize=(r1,c1))
                plt.rcParams.update({'font.size': f1})
                plt.plot(history.history['accuracy'],linewidth=linew1)
                plt.plot(history.history['val_accuracy'],linewidth=linew1)
                plt.title('Model Accuracy')
                plt.xlabel('epoch')
                plt.ylabel('accuracy')
                plt.legend(['train','test'])
            #    plt.show()
                plt.savefig(head1+'/b/acc_'+tail1+'_'+modelstr2+'.png')
                # np.save(head1+'\\acc_'+tail1+'_'+modelstr2+'.npy', history.history['accuracy'])
                # np.save(head1+'\\acc_val_'+tail1+'_'+modelstr2+'.npy', history.history['val_accuracy'])
            #    plt.clf()
                print(head1+'/aloss_'+tail1+'_'+modelstr2+'.png')

                # y_pred = model.predict(x_test, batch_size = batch_size1)
                y_pred = model.predict(x_test)
                y_test1= np.argmax(y_test, axis=1)
            #    y_test1=y_test1.tolist()
                y_pred1 = np.argmax(y_pred, axis=1)
            #    y_pred1 = y_pred1.tolist()
                # predict probabilities for test set
                yhat_probs = y_pred
                # predict crisp classes for test set
                # yhat_classes = model.predict_classes(x_test, verbose=0)
                predict_x=model.predict(x_test)
                yhat_classes=np.argmax(predict_x,axis=1)

                # reduce to 1d array
                yhat_probs = yhat_probs[:, 0]
            #    yhat_classes = yhat_classes[:, 0]
                # accuracy: (tp + tn) / (p + n)
                accuracy = accuracy_score(y_test1, yhat_classes)
                print('Accuracy: %f' % accuracy)
                # balanced accuracy: (tp + tn) / (p + n)
                b_accuracy = balanced_accuracy_score(y_test1, yhat_classes)
                print('Balanced Accuracy: %f' % b_accuracy)
                # precision tp / (tp + fp)
                precision = precision_score(y_test1, yhat_classes,average='weighted')
                print('Precision: %f' % precision)
                # recall: tp / (tp + fn)
                recall = recall_score(y_test1, yhat_classes,average='weighted')
                print('Recall: %f' % recall)
                # f1: 2 tp / (2 tp + fp + fn)
                f1 = f1_score(y_test1, yhat_classes,average='weighted')
                print('F1 score: %f' % f1)
                # kappa
                kappa = cohen_kappa_score(y_test1, yhat_classes)
                print('Cohens kappa: %f' % kappa)
            #    ROC AUC
            #    auc = roc_auc_score(y_test1, yhat_probs)
            #    print('ROC AUC: %f' % auc)
            #    confusion matrix
                conf_mat = confusion_matrix(y_test1, yhat_classes)
                df=pd.DataFrame(data=conf_mat[0:,0:], index=[i for i in range(conf_mat.shape[0])], columns=['f'+str(i) for i in range(conf_mat.shape[1])])
                df.to_excel(head1+'/b/aconf_mat_'+tail1+'_'+modelstr2+'.xlsx')
                print(conf_mat)
            #        np.savetxt(head1+'\\conf_mat_'+tail1+'_'+modelstr1+'.csv', conf_mat, '%s', delimiter=",")
                alist.loc[i]=[str1,tail1+'_'+modelstr2,'',score[1],score[0],precision,recall,f1,kappa,b_accuracy]
                i=i+1
            #    np.savetxt(head1+'\\alist_'+'_'+modelstr1+'.csv', alist, '%s', delimiter="/t")
                del x_train
                del x_val
                del x_test
                best_acc3=0
                best_acc4=0
                z1=z1+1
                if (z1%6==0):
                    alist.to_excel(head1+'/b/alist_'+'_'+tail1+'_'+modelstr2+'.xlsx')

            alist.to_excel(head1+'/b/alist_'+'_'+tail1+'_'+modelstr2+'.xlsx')

/content/gdrive/My Drive/carfac/a/1  2  4  5  7  8  9_3  6_nemo train carfac_emp3 _d __cons_512_256_0.3__f0.npy
0
1  2  4  5  7  8  9_3  6_nemo test carfac_emp3 _d __cons_512_256_0.3__f0.npy
x_train shape: (3685, 2904)
3685 train samples
x_test shape: (796, 2904)
y_test shape: (796,)
Epoch 1/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 46s 390ms/step - accuracy: 0.1852 - loss: 1.7822 - val_accuracy: 0.3894 - val_loss: 1.4135 - learning_rate: 1.0000e-04
Epoch 2/50
45/93 ━━━━━━━━━━━━━━━━━━━━ 17s 361ms/step - accuracy: 0.4052 - loss: 1.3544